# Projeto 2 - Ciência dos Dados

Nome: Luis Filipe Carrete

Nome: Manuel Castanares

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [1]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [2]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [88]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [97]:
#Produto escolhido:
produto = 'windows 10'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [100]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [101]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [3]:
dados_raw = pd.read_excel('windows 10.xlsx')

In [4]:
dados_raw.head()

,Treinamento,Relevancia
0,rt @jgaitpro: incluir políticas de grupo para ...,0
1,hora de melhorar o windows 10! chegaram os pow...,1
2,atualização do #windows 10 está fazendo a cpu ...,1
3,windows 10: nova atualização da versão 1903 po...,1
4,atualização do windows 10 estaria sobrecarrega...,1


In [5]:
dados_rel = dados_raw.loc[(dados_raw.Relevancia == 1)]

dados_irre = dados_raw.loc[(dados_raw.Relevancia == 0)]

In [6]:
import re

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    import string
    punctuation = '[!-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed

In [7]:
lista_relevante = []
for i in dados_rel.Treinamento:
    a = i.split()
    for b in a:
        if b != 'rt' and b[0:5] != "https" and b[0:1] != "@":
            c = cleanup(b)
            lista_relevante.append(c)
    
relevante = pd.Series(lista_relevante)

palavras_rel = relevante.value_counts(True)


In [8]:
lista_irrelevante = []
for i in dados_irre.Treinamento:
    a = i.split()
    for b in a:
        if b != 'rt' and b[0:5] != "https" and b[0:1] != "@":
            c = cleanup(b)
            lista_irrelevante.append(c)

irrelevante = pd.Series(lista_irrelevante)

palavras_irre = irrelevante.value_counts(True)


In [9]:
todas_palavras = []
for i in dados_raw.Treinamento:
    
    a = i.split()
    
    for b in a:
        if b != 'rt' and b[0:5] != "https" and b[0:1] != "@":
            c = cleanup(b)
            todas_palavras.append(c)

palavras_frequencia = pd.Series(todas_palavras)

palavras = palavras_frequencia.value_counts(True)



___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [10]:
tweet_teste = "rip memoria ram morre após menina criar vm no windows 10"

In [11]:
def classificador(tweet):
    i = tweet.split()
    p_rel = 1
    p_irre = 1
    for a in i:
        if a != 'rt' and a[0:5] != "https" and a[0:1] != "@":
            c = cleanup(a)
            if c in palavras_rel:
                p_rel *= palavras_rel[c]
            if c in palavras_irre:
                p_irre *= palavras_irre[c]

    if p_rel > p_irre:
        return(1)
    else:
        return(0)
    


In [12]:
print(classificador(tweet_teste))

1


___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

In [13]:
dados_teste = pd.read_excel('windows 10.xlsx', sheetname="Teste")

for tweet in dados_teste.Teste:
    dados_teste.loc[(dados_teste.Teste == tweet),"Classificador"] = [classificador(tweet)]
    

C:\Users\lfcsa\Anaconda3\lib\site-packages\pandas\io\excel.py:329: FutureWarning: The `sheetname` keyword is deprecated, use `sheet_name` instead
  **kwds)


In [15]:
dados_teste

,Teste,Relevancia,Classificador
0,como usar os powertoys do windows 10 https://t...,0,1.0
1,"tô sem pc, posso nem ver minhas seriezinhas ☹️...",1,0.0
2,rip memoria ram morre após menina criar vm no ...,1,1.0
3,"@renao @lothermus é merda até hoje, acho que n...",1,0.0
4,rt @rafaelgrn: dead cells saiu com uma nova li...,0,0.0
5,windows 10 nem faz barulho pra ligar porra htt...,1,0.0
6,downtime acontece exatamente durante o fim de ...,0,1.0
7,na moral windows 10 foi a pior invenção,1,0.0
8,📣‍super oferta #amazon | #informática\nmedion...,0,1.0
9,"@windowsbr eu quero que, pelo amor de deus, pa...",1,0.0


___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**